In [1]:
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, MaxPooling2D, Dropout, BatchNormalization, Reshape
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import CosineDecay
from keras import regularizers
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler



# prompt: how to include a google sheets in here?
creds, _ = default()
gc = gspread.authorize(creds)
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1Zhn2sKtMYfRU9rTyV8xlc9CrX7Q3274QWJL2MyyGDSg/edit?usp=sharing'
doc = gc.open_by_url(spreadsheet_url)
worksheet = doc.worksheet('Sheet2')
# Access data from the spreadsheet
data = worksheet.get_all_values()


# Memory buffer to store recently matched elements
memory_buffer = []

######################################################################################################

# Print the data
def Printo(list_of_numbers):

  for number in list_of_numbers:
    print(number)
"""
        This function prints each number in a list of numbers.
"""
######################################################################################################

# convert data to int
def convert_to_int(data):
    return [[int(element) for element in sublist] for sublist in data]
"""
        This function converts each element in a list of lists to integers.
"""
######################################################################################################

# create a grid
def create_grid(data):
    grids = []

    for sublist in data:
        # Initialize a 10x5 grid with '.' values
        grid = [[0 for _ in range(17)] for _ in range(3)]

        for number in sublist:
            if 1 <= number <= 50:
                # Calculate the row and column based on the provided pattern
                col = (number - 1) % 17 # 0, 1, 2
                row = (number - 1) // 17 # 0, 1, 2, ... , 16
                grid[row][col] = 1

        grids.append(grid)

    return grids
"""
            This function creates a 3x17 grid from a list of numbers.
"""
######################################################################################################


def reverse_grid(grid):

    # Ensure grid is a list of grids
    if isinstance(grid, np.ndarray) and grid.shape == (3, 17):
        grids = [grid]  # Wrap single grid in a list
    else:
        grids = grid

    reversed_data = []

    for grid in grids:
        sublist = []
        # Iterate through each cell in the 3x17 grid
        for row in range(3):
            for col in range(17):
                if grid[row][col] == 1:
                    # Calculate the corresponding number (1 to 50)
                    number = row * 17 + col + 1
                    sublist.append(number)

        reversed_data.append(sublist)

    return reversed_data if len(reversed_data) > 1 else reversed_data[0]

"""
            Reverses the create_grid function by converting each 3x17 grid back into a list of numbers.

            Parameters:
            grids (list): A list of grids, where each grid is a 3x17 binary matrix.

            Returns:
            list: A list of sublists, where each sublist contains numbers (1 to 50) corresponding to the positions of 1s in the grid.
"""
######################################################################################################

def blend_weights_for_targets(best_weights, current_weights, dynamic_alpha=0.5):

    blended_weights = []

    # Blend weights for the layers based on the priority of recent predictions
    for current_w, best_w in zip(current_weights, best_weights):
        # You can give dynamic_alpha based on prediction priority
        blended_w = dynamic_alpha * current_w + (1 - dynamic_alpha) * best_w
        blended_weights.append(blended_w)

    return blended_weights
"""
            Blends the best and current weights for elements that were correctly predicted (targets).

            Arguments:
            - best_weights: Best model weights so far.
            - current_weights: Current model weights.
            - matched_elements: Set of elements that were correctly predicted.
            - dynamic_alpha: Adjusted blending factor to give priority to recent guesses.

            Returns:
            - blended_weights: Blended weights with priority to recent correct predictions.
"""
######################################################################################################

# Define the learning rate schedule
def lr_schedule(epoch):
    initial_lr = 0.0023
    decay_rate = 0.0001
    decay_steps = 10
    lr = initial_lr * decay_rate ** (epoch / decay_steps)
    return lr
######################################################################################################


In [2]:
################################################################################
data = convert_to_int(data)
data_panda = pd.DataFrame(np.array(data),columns=list('ABCDEFGX'))
print(data_panda)
#data panda is the data not normalized, Pandas DataFram, written numbers array
################################################################################
grids = create_grid(data)
grids_np = np.array(grids)
print(type(grids_np))
print(grids_np.shape)
print(grids_np[0])
print(grids_np[1])
#grids_np is a grid of numbers in the shape of the 3 * 17 scan-tron, 0 for loser and 1 for winner 10
################################################################################
data_canda = data_panda / 50.0000
#data_canda.replace(0.02, 0, inplace=True)
#scaler = MinMaxScaler(feature_range=(0, 1))
print(data_canda)
#data canda is the data normalized, Pandas DataFram, written numbers array
################################################################################

      A   B   C   D   E   F   G   X
0     2   4  11  16  25  29  47  34
1     9  12  13  15  29  30  41  47
2     5  16  22  24  31  33  38  30
3     5  16  18  20  23  28  39   8
4     4   6  10  22  32  39  50  19
...  ..  ..  ..  ..  ..  ..  ..  ..
1057  4   7  10  12  23  25  36  18
1058  4  10  18  20  31  35  44  29
1059  5  15  25  38  42  46  47  26
1060  8  27  28  29  31  32  35  11
1061  5  17  19  25  31  38  46   4

[1062 rows x 8 columns]
<class 'numpy.ndarray'>
(1062, 3, 17)
[[0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]]
[[0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0]]
         A     B     C     D     E     F     G     X
0     0.04  0.08  0.22  0.32  0.50  0.58  0.94  0.68
1     0.18  0.24  0.26  0.30  0.58  0.60  0.82  0.94
2     0.10  0.32  0.44  0.48  0.62  0.66  0.76  0.60
3     0.10  0.32  0.36  0.40  0.46  0.56  0.78  0.16
4     0.08  0.12

In [3]:

def get_top_8_binary_matrix(predicted_data):

    # Flatten the predicted_data to easily work with indices
    flat_data = predicted_data.flatten()

    # Get the indices of the top 8 highest values
    top_8_indices = np.argpartition(flat_data, -8)[-8:]

    # Create an empty binary matrix (all zeros) with the same shape as the predicted_data
    binary_matrix = np.zeros_like(flat_data)

    # Set the top 8 indices to 1
    binary_matrix[top_8_indices] = 1

    # Reshape the binary matrix back to the original shape (3, 17 in this case)
    binary_matrix = binary_matrix.reshape(predicted_data.shape)

    return binary_matrix
"""
    Takes the predicted data, identifies the 8 highest elements,
    and returns a binary matrix where those positions are marked as 1.

    Parameters:
    predicted_data (numpy.ndarray): The predicted data, typically of shape (3, 17).

    Returns:
    numpy.ndarray: A binary matrix of the same shape as predicted_data, with the 8 highest elements set to 1.
"""
######################################################################################################


'\n    Takes the predicted data, identifies the 8 highest elements,\n    and returns a binary matrix where those positions are marked as 1.\n\n    Parameters:\n    predicted_data (numpy.ndarray): The predicted data, typically of shape (3, 17).\n\n    Returns:\n    numpy.ndarray: A binary matrix of the same shape as predicted_data, with the 8 highest elements set to 1.\n'

In [4]:
def get_top_8_binary_matrix_with_rankings(predicted_data):

    # Flatten the predicted_data to easily work with indices and probabilities
    flat_data = predicted_data.flatten()

    # Get the indices of the top 8 highest values
    top_8_indices = np.argpartition(flat_data, -8)[-8:]

    # Create an empty binary matrix (all zeros) with the same shape as the predicted_data
    binary_matrix = np.zeros_like(flat_data)

    # Set the top 8 indices to 1 in the binary matrix
    binary_matrix[top_8_indices] = 1

    # Reshape the binary matrix back to the original shape (3, 17 in this case)
    binary_matrix = binary_matrix.reshape(predicted_data.shape)

    # Get the corresponding probabilities for the top 8 indices
    top_8_probabilities = flat_data[top_8_indices]

    # Create a list of tuples with (index, probability), sorted by probability in descending order
    ranked_top_8 = sorted(
        [(idx, flat_data[idx]) for idx in top_8_indices],
        key=lambda x: x[1],  # Sort by probability
        reverse=True
    )

    return binary_matrix, ranked_top_8
"""
    Takes the predicted data, identifies the 8 highest elements,
    and returns a binary matrix where those positions are marked as 1,
    along with their rankings (sorted probabilities).

    Parameters:
    predicted_data (numpy.ndarray): The predicted data, typically of shape (3, 17).

    Returns:
    tuple:
    - numpy.ndarray: A binary matrix of the same shape as predicted_data, with the 8 highest elements set to 1.
    - list of tuples: Each tuple contains (index, probability) sorted by probability (highest first).
"""
######################################################################################################


'\n    Takes the predicted data, identifies the 8 highest elements,\n    and returns a binary matrix where those positions are marked as 1,\n    along with their rankings (sorted probabilities).\n\n    Parameters:\n    predicted_data (numpy.ndarray): The predicted data, typically of shape (3, 17).\n\n    Returns:\n    tuple:\n    - numpy.ndarray: A binary matrix of the same shape as predicted_data, with the 8 highest elements set to 1.\n    - list of tuples: Each tuple contains (index, probability) sorted by probability (highest first).\n'

In [5]:
def update_memory_buffer(matching_elements, buffer_size=8):

    global memory_buffer
    for element in matching_elements:
        if element not in memory_buffer:
            memory_buffer.append(element)
        # Keep the buffer size constrained
        if len(memory_buffer) > buffer_size:
            memory_buffer.pop(0)
"""
    Update the memory buffer with matching elements.
    Ensures that the buffer keeps the most recent successful elements (up to buffer_size).
"""
######################################################################################################
def prioritize_memory_weights(current_weights, best_weights, dynamic_alpha=0.5):

    blended_weights = []
    for i in range(len(current_weights)):
        if i in memory_buffer:
            # Prioritize memory elements with higher weight
            blended_weights.append((dynamic_alpha * best_weights[i]) + ((1 - dynamic_alpha) * current_weights[i]))
        else:
            # Regular blending for non-memory elements
            blended_weights.append((0.5 * best_weights[i]) + (0.5 * current_weights[i]))
    return blended_weights
"""
    Blend weights by prioritizing elements in the memory buffer.
    If an element is found in memory, its corresponding weight is given higher priority.
"""
######################################################################################################

'\n    Blend weights by prioritizing elements in the memory buffer.\n    If an element is found in memory, its corresponding weight is given higher priority.\n'

In [6]:
from re import A
# The main training loop with memory integration
def train_until_match(model, X, y, N, target_list, alpha=0.5, buffer_size=8):
    lr_scheduler = ReduceLROnPlateau(monitor='loss', mode='auto', factor=0.001, min_lr=0.0018)
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=False)

    best_match_percentage = 0
    best_weight = None
    best_index = 0
    alreadyMATCH_set = set()
    epk = 25
    n = 6

    for i in range(N):
        # Train the model
        model.fit(
            X, y.reshape((y.shape[0], -1)),
            epochs=epk,
            batch_size=n,
            validation_split=0.1,
            shuffle=False,
            callbacks=[early_stopping, lr_scheduler]
        )
        # Predict the next data unit
        predicted_next_data = model.predict(y[0].reshape(1, 3, 17, 1))
        predicted_next_data = predicted_next_data.reshape((3, 17))  # Reshape the predicted output

        # Convert the prediction to a binary matrix based on the top 8 values
        binary_matrix, ranked_top_8 = get_top_8_binary_matrix_with_rankings(predicted_next_data)
        print(f"Iteration {i + 1} - Binary matrix:")
        print(binary_matrix)
        # Print the rankings of the top 8 selected probabilities
        print("Ranked top 8 (index, probability):")
        for idx, prob in ranked_top_8:
          prob = prob * 100
          print(f"Index: {idx + 1}, Probability: {prob:.2f}%")

        # Convert the binary matrix to a list of numbers
        numbers_list = reverse_grid(binary_matrix)
        print(f"Iteration {i + 1} - Predicted numbers list:", numbers_list)

        # Compare the predicted result with the target list
        if numbers_list == target_list:
            print(f"Match found at iteration {i + 1}!")
            best_weight = model.get_weights()  # Save the best model weights
            update_memory_buffer(set(numbers_list))  # Update memory buffer with full match
            break
        else:
            matching_elements = set(numbers_list).intersection(set(target_list))
            match_percentage = (len(matching_elements) / len(target_list)) * 100
            unique_elements = matching_elements - alreadyMATCH_set
            if match_percentage > best_match_percentage:
              print("1.Percentage Increase")
              # PERCENTAGE
              best_match_percentage = match_percentage
              # INDEX
              best_index = i + 1
              # WEIGHTS
              best_weight = model.get_weights()  # Save best weights #model.set_weights(best_weight)  # Set blended/best weights
              # UPDATE MEM AND SET
              alreadyMATCH_set.update(matching_elements)
              model.set_weights(best_weight)  # Set the weights to the model
              print(f"1.111.Results at iteration {i+1}. Current Match Percentage: {match_percentage}%. Matches found: {matching_elements}.")
              print(f"1.1111.Best match found at iteration {best_index}. Best Match Percentage: {best_match_percentage}%. Matches found Previously: {alreadyMATCH_set}. ")

            elif match_percentage == best_match_percentage:
              print("2.Percentage Remains the Same")
              if unique_elements:
                print("2.1.Combine Weights Since there are matches never seen before!")
                alreadyMATCH_set.update(matching_elements)
                # do something so next time during training, the unique match is added to matched_elements (requires weight manuplaition), and the best_match_percentage increased.
              else:
                if matching_elements != alreadyMATCH_set:
                  print("2.2.Combine Weights Since there are matches never seen before!")
                  alreadyMATCH_set.update(matching_elements)
                  # do something so next time during training, the unique match is added to matched_elements (requires weight manuplaition), and the best_match_percentage increased.

                else:
                  print("2.9.Do Nothing")
              print(f"2.222.Results at iteration {i+1}. Current Match Percentage: {match_percentage}%. Matches found: {matching_elements}.")
              print(f"2.2222.Best match found at iteration {best_index}. Best Match Percentage: {best_match_percentage}%. Matches found Previously: {alreadyMATCH_set}. ")


            elif match_percentage < best_match_percentage:
                print("3.Percentage Decreases")
                if unique_elements:
                    print("3.1.Combine Weights Since there are matches never seen before!")
                    alreadyMATCH_set.update(matching_elements)
                    # do something so next time during training, the unique match is added to matched_elements (requires weight manuplaition), and the best_match_percentage increased.

                else:
                    print("3.9.Do Nothing")
                print(f"3.333.Results at iteration {i+1}. Current Match Percentage: {match_percentage}%. Matches found: {matching_elements}.")
                print(f"3.3333.Best match found at iteration {best_index}. Best Match Percentage: {best_match_percentage}%. Matches found Previously: {alreadyMATCH_set}. ")
            print(f"#########################No full match found at iteration {i + 1}.#########################")

    else:
        print(f"#########################No full match found after {N} iterations.#########################")
        print(f"Best match found at iteration {best_index}, with {best_match_percentage:.2f}% match rate.")
        model.set_weights(best_weight)  # Set the weights to the model
    return best_weight


In [7]:

# Assuming data is your numpy array of shape (1061, 3, 17)
data = grids_np  # Example data; replace with your actual dataset

# Prepare the input and output data
X = data[1:]  # All data except the last one (1060 samples)
y = data[:-1]   # All data except the first one (1060 samples)
print(type(X))
print(y.shape)


# Reshape data to add a channel dimension, which is required for Conv2D (e.g., (1060, 3, 17, 1))
X = X.reshape((X.shape[0], X.shape[1], X.shape[2], 1))
y = y.reshape((y.shape[0], y.shape[1], y.shape[2], 1))


# Define the CNN model
model = Sequential()
model.add(Input(shape=(3, 17, 1)))
model.add(Conv2D(filters=512, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(1, 2)))  # Pooling to reduce dimensions
#model.add(Dropout(0.2))  # Add dropout to prevent overfitting
model.add(Conv2D(filters=128, kernel_size=(2, 2), activation='relu'))
model.add(Flatten())  # Flatten the output from the convolutional layers
model.add(Dense(units=51, activation='relu'))  # Fully connected layer
model.add(Dense(units=51, activation='relu'))  # Fully connected layer
model.add(Dense(units=51, activation='relu'))  # Fully connected layer

model.add(Dense(units=3 * 17, activation='sigmoid'))  # Output layer: 3x17 units
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.002), loss='binary_crossentropy', metrics=['accuracy'])



<class 'numpy.ndarray'>
(1061, 3, 17)


In [9]:
# Example usage
N = 10  # Number of iterations to run
############################################################ CHANGE THIS
target_list = [4, 8, 26, 32, 35, 36, 37, 33]  # Target list to match against
############################################################ CHANGE THIS
best_weight = train_until_match(model, X, y, N, target_list)
# Assume model, X, y, early_stopping, and lr_scheduler are already defined


Epoch 1/25
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.0198 - loss: 0.4326 - val_accuracy: 0.0561 - val_loss: 0.4323 - learning_rate: 0.0020
Epoch 2/25
159/159 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.0222 - loss: 0.4328 - val_accuracy: 0.0187 - val_loss: 0.4315 - learning_rate: 0.0020
Epoch 3/25
159/159 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.0384 - loss: 0.4320 - val_accuracy: 0.0187 - val_loss: 0.4329 - learning_rate: 0.0020
Epoch 4/25
159/159 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.0282 - loss: 0.4311 - val_accuracy: 0.0280 - val_loss: 0.4341 - learning_rate: 0.0020
Epoch 5/25
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.0359 - loss: 0.4299 - val_accuracy: 0.0093 - val_loss: 0.4368 - learning_rate: 0.0020
Epoch 6/25
159/159 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.0439 - loss: 0.4282 - val_accuracy: 0.0374 - val_loss: 0.4383 - learning_rate: 0.0020
Epoch 7/25
159/159 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.0520 - loss: 0.

In [15]:
# Predict the next data unit
#print(y[0])
#print(y[1])
from google.colab import drive
drive.mount('/content/drive')

"""model.fit(
    X, y.reshape((y.shape[0], -1)),
    epochs=50,  # Set a large number of epochs; early stopping will handle stopping earlier if needed
    batch_size=16,  # You can adjust the batch size as needed
    validation_split=0.0,  # Reserve a portion of the data for validation
    callbacks=[early_stopping, lr_scheduler]
)
"""
# use the best weights to predict the next one input current
model.set_weights(best_weight)
model.save_weights('/content/drive/My Drive/modelseptember20_2024_50p.weights.h5')
# Predict the next data unit
#print(y[0]
predicted_next_data = model.predict(y[0].reshape(1, 3, 17, 1))

# Reshape the predicted output back to (3, 17)
predicted_next_data = predicted_next_data.reshape((3, 17))

#print("Predicted next data unit:\n", predicted_next_data)


#binary_matrix = (predicted_next_data >= 0.5).astype(int) #0.4684 ~ 0.4685
binary_matrix = get_top_8_binary_matrix(predicted_next_data).astype(int)

print("Best attempt at Cutrrent One:")
print(binary_matrix)
#print(type(binary_matrix))
#print(binary_matrix.shape)
numbers_list = reverse_grid(binary_matrix)
print(numbers_list)


#target_list = [2, 4, 11, 16, 25, 29, 34, 47]  # Target list to match against

target = np.array([[
    [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0],
    [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
]])
############################################################ CHANGE THIS
#print(target.shape)
predicted_next_data_A = model.predict(target.reshape(1, 3, 17, 1))
#print(predicted_next_data_A)
binary_matrix = get_top_8_binary_matrix(predicted_next_data_A).astype(int)
binary_matrix = binary_matrix.reshape(3,17)
print("Next One:")
print(binary_matrix)
numbers_list = reverse_grid(binary_matrix)
print(numbers_list)



Mounted at /content/drive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Best attempt at Cutrrent One:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 0 0 0 1 0 1 1 0]
 [0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
[18, 19, 21, 30, 32, 33, 36, 37]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Next One:
[[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 1 0]]
[10, 18, 21, 30, 37, 42, 43, 50]


In [12]:
for layer in model.layers: print(layer.get_config(), layer.get_weights())


{'name': 'conv2d', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'filters': 512, 'kernel_size': (2, 2), 'strides': (1, 1), 'padding': 'valid', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'groups': 1, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None} [array([[[[-4.9295254e-02, -2.6059765e-02, -1.1258733e-01, ...,
           3.9215479e-02,  3.0074757e-01, -4.4591581e-03]],

        [[-2.0186869e-02,  3.1822303e-05,  1.3120140e-02, ...,
          -1.6217667e-01, -2.8287616e-01, -1.7328547e-02]]],


       [[[-9.78753